In [41]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
from keras.preprocessing.image import ImageDataGenerator,load_img
import cv2
from random import shuffle
from shutil import copyfile,rmtree
import json
from imgaug import augmenters as iaa
import numpy as np
from PIL import Image

class ImagePrep:
    
    def getTrainTest(self,directory,extention):
        files = []
        for file in os.listdir(directory):
            if file.endswith(extention):
                files.append(file)
        shuffle(files)
        splt = int(len(files)*0.8)
        x_train_files = files[0:splt]
        x_test_files = files[splt:-1]
        
        json_data = open(directory +'/calories_json.json',mode='r').read()
        y_map = json.loads(json_data)
        
        x_train = [load_img(directory+"/"+x, target_size=(224,224,3)) for x in x_train_files]
        y_train = [y_map[x] for x in x_train_files]
        x_test = [load_img(directory+"/"+x, target_size=(224,224,3)) for x in x_test_files]
        y_test = [y_map[x] for x in x_test_files]
        
        return x_train,y_train,x_test,y_test
    
    def augmentImages(self,x,y):
        x_train_aug = []
        y_train_aug = []
        
        for i in range(len(x)):
            x_train_aug.append(x[i])
            y_train_aug.append(y[i])
            
            #transformation one - adding random black dots in the image
            t = iaa.CoarseDropout(0.1, size_percent=0.2).augment_images([np.array(x[i])])
            x_train_aug.append(Image.fromarray(np.uint8(t[0])))
            y_train_aug.append(y[i])
            
            #transformation two - simple rotation
            t = iaa.Affine(rotate=(-25, 25)).augment_images([np.array(x[i])])
            x_train_aug.append(Image.fromarray(np.uint8(t[0])))
            y_train_aug.append(y[i])
        
            #transformation three - increasing the contrast of the objects in the image
            t = iaa.GammaContrast(1.5).augment_images([np.array(x[i])])
            x_train_aug.append(Image.fromarray(np.uint8(t[0])))
            y_train_aug.append(y[i])
            
            #transformation four - mirror effect
            t = iaa.Fliplr(p = 1.0).augment_images([np.array(x[i])])
            x_train_aug.append(Image.fromarray(np.uint8(t[0])))
            y_train_aug.append(y[i])
            
            #transformation five - blurring effect
            t = iaa.GaussianBlur((0, 3.0)).augment_images([np.array(x[i])])
            x_train_aug.append(Image.fromarray(np.uint8(t[0])))
            y_train_aug.append(y[i])
            
            #transformation six - cropping and padding, expand, squish, move effect
            t = iaa.CropAndPad(percent=(-0.25, 0.25)).augment_images([np.array(x[i])])
            x_train_aug.append(Image.fromarray(np.uint8(t[0])))
            y_train_aug.append(y[i])
            
            #transformation seven - it randomly moves pixels around within a given bound
            t = iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25).augment_images([np.array(x[i])])
            x_train_aug.append(Image.fromarray(np.uint8(t[0])))
            y_train_aug.append(y[i])
            
        return x_train_aug,y_train_aug

In [0]:
obj = ImagePrep()
x_train,y_train,x_test,y_test = obj.getTrainTest("drive/My Drive/subway",'.png')
x_train_aug,y_train_aug = obj.augmentImages(x_train,y_train)

x_train_aug_np = np.array([np.array(x) for x in x_train_aug])
y_train_aug = np.array(y_train_aug).astype('int')

In [0]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam
from keras.losses import mean_squared_error as mse
from keras import layers,models
from keras.applications.resnet50 import ResNet50

class Models:

    def cnn(
        self,
        width,
        height,
        depth,
        filters=(16, 32, 64),
        regress=False,
        ):

        # initialize the input shape and channel dimension, assuming
        # TensorFlow/channels-last ordering

        inputShape = (height, width, depth)
        chanDim = -1

        # define the model input

        inputs = Input(shape=inputShape)

        # loop over the number of filters

        for (i, f) in enumerate(filters):

            # if this is the first CONV layer then set the input
            # appropriately

            if i == 0:
                x = inputs

            # CONV => RELU => BN => POOL

            x = Conv2D(f, (3, 3), padding='same')(x)
            x = Activation('relu')(x)
            x = BatchNormalization(axis=chanDim)(x)
            x = MaxPooling2D(pool_size=(2, 2))(x)

            # flatten the volume, then FC => RELU => BN => DROPOUT

        x = Flatten()(x)
        x = Dense(16)(x)
        x = Activation('relu')(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = Dropout(0.5)(x)

        # apply another FC layer, this one to match the number of nodes
        # coming out of the MLP

        x = Dense(4)(x)
        x = Activation('relu')(x)

        # check to see if the regression node should be added

        if regress:
            x = Dense(1, activation='linear')(x)

        # construct the CNN

        model = Model(inputs, x)

        # return the CNN

        return model

    def resnet(self, freezing):
        resnet50_model = ResNet50(include_top=True, weights='imagenet')

        # in this model we are freezing layers(3,116) inclusive freezing

        if freezing:
            times = 0
        for i in range(3, 200, 1):
            if 'BatchNormalization' \
                in str(type(resnet50_model.layers[i])) and times == 40:
                break
            else:
                if 'BatchNormalization' \
                    in str(type(resnet50_model.layers[i])):
                    times += 1
            resnet50_model.layers[i].trainable = False
        regression_model = models.Sequential()
        resnet50_model.layers[176] = layers.Dense(512, activation='relu'
                )
        regression_model.add(resnet50_model)
        regression_model.add(layers.Dense(256, activation='sigmoid'))
        regression_model.add(layers.Dense(128, activation='relu'))
        regression_model.add(layers.Dense(1, activation='linear'))
        regression_model.compile(optimizer='sgd',
                                 loss='mean_squared_error',
                                 metrics=['accuracy'])
        return regression_model


In [0]:
models = Models()
cnn_model = models.cnn(224,224,3,regress=True)

In [0]:
from keras.optimizers import Adam
opt = Adam(lr=1e-3, decay=1e-3 / 200)
cnn_model.compile(loss="mean_absolute_percentage_error", optimizer=opt)

In [52]:
cnn_model.fit(x_train_aug_np, y_train_aug, epochs=20, batch_size=8)

Epoch 1/20
960/960 [==============================] - 4s 4ms/step - loss: 48.1403
Epoch 2/20
960/960 [==============================] - 4s 4ms/step - loss: 47.2358
Epoch 3/20
960/960 [==============================] - 4s 4ms/step - loss: 45.4547
Epoch 4/20
960/960 [==============================] - 4s 4ms/step - loss: 44.7804
Epoch 5/20
960/960 [==============================] - 4s 4ms/step - loss: 43.1818
Epoch 6/20
960/960 [==============================] - 4s 4ms/step - loss: 40.2223
Epoch 7/20
960/960 [==============================] - 4s 4ms/step - loss: 42.2194
Epoch 8/20
960/960 [==============================] - 4s 4ms/step - loss: 39.6404
Epoch 9/20
960/960 [==============================] - 4s 4ms/step - loss: 37.6669
Epoch 10/20
960/960 [==============================] - 4s 4ms/step - loss: 37.0169
Epoch 11/20
960/960 [==============================] - 4s 4ms/step - loss: 35.0587
Epoch 12/20
960/960 [==============================] - 4s 4ms/step - loss: 34.4465
Epoch 13/20
9

In [0]:
x_test_np = np.array([np.array(x) for x in x_test])
y_pred = cnn_model.predict(x_test_np)

In [54]:
y_test = np.array(y_test).astype('int')
print(np.average(y_pred.flatten() - y_test))
print(y_pred.flatten())
print(y_test)
print(y_test-y_pred.flatten())

8.700533040364583
[547.955   661.16376 863.71344 677.2073  686.2107  903.4587  564.3907
 750.31305 774.5853  444.19    305.08844 610.4329  732.9163  496.39307
 922.2946  604.6076  704.2955  702.2757  657.7255  362.64484 780.11
 458.68585 704.8116  572.0615  552.23376 663.2758  796.8928  777.4291
 488.2895  485.36365]
[ 370 1200  950  390  790  740  320 1090  740  630  220  430 1160  920
  600  310  570  780  380  200  890  540  600  440  380  450  650  600
  400 1250]
[-177.95501709  538.83624268   86.28656006 -287.20727539  103.78930664
 -163.4586792  -244.39068604  339.68695068  -34.58532715  185.80999756
  -85.08843994 -180.43292236  427.0836792   423.60693359 -322.2946167
 -294.60760498 -134.29547119   77.7243042  -277.7255249  -162.64483643
  109.89001465   81.31414795 -104.81158447 -132.06152344 -172.23376465
 -213.27581787 -146.89282227 -177.42907715  -88.28948975  764.63635254]
